### Inference Engine Building

In [1]:
pip install pysimplegui

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import PySimpleGUI as sg

from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from PIL import Image, ImageTk

In [3]:
df = pd.read_csv("calories_decision_table.csv")

df.head()

,Age,BMI,Sex,Activity_intensity,Calories_need
0,<17,"<18,5",M,High,2170
1,17-29,"<18,5",M,High,2101
2,30-50,"<18,5",M,High,1957
3,>50,"<18,5",M,High,1787
4,<17,"<18,5",F,High,1887


In [4]:
df.shape

(120, 5)

In [5]:
df.nunique()

Age                     4
BMI                     5
Sex                     2
Activity_intensity      3
Calories_need         118
dtype: int64

In [6]:
print(f"Unique Age: {df['Age'].unique()}")
print(f"Unique BMI: {df['BMI'].unique()}")
print(f"Unique Sex: {df['Sex'].unique()}")
print(f"Unique Activity_intensity: {df['Activity_intensity'].unique()}")

Unique Age: ['<17' '17-29' '30-50' '>50']
Unique BMI: ['<18,5' '>35' '18,5-24,9' '25-29,9' '30-34,9']
Unique Sex: ['M' 'F']
Unique Activity_intensity: ['High' 'Medium' 'Low']


In [7]:
inputs = df.drop('Calories_need', axis = 'columns')
target = df['Calories_need']

In [8]:
inputs['age_n'] = df['Age'].replace(['<17', '17-29', '30-50', '>50'], [0, 1, 2, 3])
inputs['bmi_n'] = df['BMI'].replace(['<18,5', '18,5-24,9', '25-29,9', '30-34,9', '>35'], [0, 1, 2, 3, 4])
inputs['sex_n'] = df['Sex'].replace(['M', 'F'], [0, 1])
inputs['activity_n'] = df['Activity_intensity'].replace(['High', 'Medium', 'Low'], [0, 1, 2])

inputs.head()

,Age,BMI,Sex,Activity_intensity,age_n,bmi_n,sex_n,activity_n
0,<17,"<18,5",M,High,0,0,0,0
1,17-29,"<18,5",M,High,1,0,0,0
2,30-50,"<18,5",M,High,2,0,0,0
3,>50,"<18,5",M,High,3,0,0,0
4,<17,"<18,5",F,High,0,0,1,0


In [9]:
inputs_n = inputs.drop(['Age', 'BMI', 'Sex', 'Activity_intensity'], axis = 'columns')

inputs_n.head()

,age_n,bmi_n,sex_n,activity_n
0,0,0,0,0
1,1,0,0,0
2,2,0,0,0
3,3,0,0,0
4,0,0,1,0


In [10]:
model = tree.DecisionTreeClassifier()

In [11]:
model.fit(inputs_n, target)

DecisionTreeClassifier()

In [12]:
model.score(inputs_n, target)

1.0

In [13]:
#dataset from https://www.kaggle.com/vaishnavivenkatesan/food-and-their-calories
food = pd.read_csv("Food_and_Calories_-_Sheet1.csv")

food.head()

,Food,Serving,Calories
0,Artichoke,1 artichoke (128 g),60 cal
1,Arugula,1 leaf (2 g),1 cal
2,Asparagus,1 spear (12 g),2 cal
3,Aubergine,1 aubergine (458 g),115 cal
4,Beetroot,1 beet (82 g),35 cal


In [14]:
food['Calories'] = food['Calories'].str.replace(" cal", "")

food.head()

,Food,Serving,Calories
0,Artichoke,1 artichoke (128 g),60
1,Arugula,1 leaf (2 g),1
2,Asparagus,1 spear (12 g),2
3,Aubergine,1 aubergine (458 g),115
4,Beetroot,1 beet (82 g),35


### Test Run

In [15]:
ageInput = 1
bmiInput = 1
sexInput = 1
activityInput = 1

caloriesOutput = model.predict([[ageInput, bmiInput, sexInput, activityInput]])[0]
caloriesOutput

1956

In [16]:
breakfast = food.iloc[(food['Calories'].astype(int) - (caloriesOutput * 0.3)).abs().argsort()[:1]]

breakfast.head()

,Food,Serving,Calories
225,Spelt,1 cup (174 g),588


In [17]:
lunch = food.iloc[(food['Calories'].astype(int) - (caloriesOutput * 0.5)).abs().argsort()[:1]]

lunch.head()

,Food,Serving,Calories
554,Veggie Patty,1 sandwich (247 g),963


In [18]:
dinner = food.iloc[(food['Calories'].astype(int) - (caloriesOutput * 0.2)).abs().argsort()[:1]]

dinner.head()

,Food,Serving,Calories
535,McDonald’s Filet-o-Fish,1 sandwich (142 g),391


In [19]:
print("INPUT")
print("Age: 17-29")
print("BMI: 18,5-24,9")
print("Sex: F")
print("Activity_intensity: Middle")
print()
print("OUTPUT")
print("Calories_need:", caloriesOutput)
print("Breakfast:", breakfast['Food'].values[0], "|", breakfast['Serving'].values[0], "|", breakfast['Calories'].values[0], "cal")
print("Lunch:", lunch['Food'].values[0], "|", lunch['Serving'].values[0], "|", lunch['Calories'].values[0], "cal")
print("Dinner:", dinner['Food'].values[0], "|", dinner['Serving'].values[0], "|", dinner['Calories'].values[0], "cal")

INPUT
Age: 17-29
BMI: 18,5-24,9
Sex: F
Activity_intensity: Middle

OUTPUT
Calories_need: 1956
Breakfast: Spelt | 1 cup (174 g) | 588 cal
Lunch: Veggie Patty | 1 sandwich (247 g) | 963 cal
Dinner: McDonald’s Filet-o-Fish | 1 sandwich (142 g) | 391 cal


### User Interface

In [20]:
def findCaloriesNeed(uAge, uWeight, uHeight, uSex, uActivity):
    if uAge < 17:
        ageInput = 0
    elif uAge >= 17 and uAge <= 29:
        ageInput = 1
    elif uAge >= 30 and uAge <= 50:
        ageInput = 2
    else:
        ageInput = 3
    
    uBMI = uWeight / (uHeight * uHeight / 10000)
    
    if uBMI < 18.5:
        bmiInput = 0
    elif uBMI >= 18.5 and uBMI < 25:
        bmiInput = 1
    elif uBMI >= 25 and uBMI < 30:
        bmiInput = 2
    elif uBMI >= 30 and uBMI < 35:
        bmiInput = 3
    else:
        bmiInput = 4
    
    if uSex == "Male":
        sexInput = 0
    else:
        sexInput = 1
    
    if uActivity == "High":
        activityInput = 0
    elif uActivity == "Medium":
        activityInput = 1
    else:
        activityInput = 2
    
    caloriesOutput = model.predict([[ageInput, bmiInput, sexInput, activityInput]])[0]
    
    return caloriesOutput

In [70]:
sg.theme('Reddit')

update_biodata = [
    [
        sg.Image(key = 'healthyIcon1', pad = ((80, 0), (0, 0))),
        sg.Image(key = 'healthy1', pad = ((0, 80), (0, 0)))
    ],
    [
        sg.Text()
    ],
    [
        sg.Text("UPDATE BIODATA", size = (38, 1), justification = 'center')
    ],
    [
        sg.Text()
    ],
    [
        sg.Text("Age", size = (15, 1)),
        sg.In(size = (25, 1), enable_events = True, key = "uAge", background_color = '#FDEFB2')
    ],
    [
        sg.Text("Weight", size = (15, 1)),
        sg.In(size = (25, 1), enable_events = True, key = "uWeight", background_color = '#FDEFB2')
    ],
    [
        sg.Text("Height", size = (15, 1)),
        sg.In(size = (25, 1), enable_events = True, key = "uHeight", background_color = '#FDEFB2')
    ],
    [
        sg.Text("Sex", size = (15, 1)),
        sg.InputCombo(['Male', 'Female'], size = (23, 1), key = "uSex", background_color = '#FDEFB2')
    ],
    [
        sg.Text("Activity Intensity", size = (15, 1)),
        sg.InputCombo(['High', 'Medium', 'Low'], size = (23, 1), key = "uActivity", background_color = '#FDEFB2', text_color = 'black')
    ],
    [
        sg.Text()
    ],
    [
        sg.Button('Generate Menu', size = (18, 1), button_color = ('black', '#F6D180')),
        sg.Button('Cancel', size = (18, 1), button_color = ('black', '#FDEFB2'))
    ]
]

todays_menu = [
    [
        sg.Image(key = 'healthyIcon2', pad = ((80, 0), (0, 0))),
        sg.Image(key = 'healthy2', pad = ((0, 80), (0, 0)))
    ],
    [
        sg.Text()
    ],
    [
        sg.Text("TODAY'S MENU", size = (38, 1), justification = 'center')
    ],
    [
        sg.Text("Total Calories Need: -" , size = (38, 1), justification = 'center', key = "dailyCalories")
    ],
    [
        sg.Text()
    ],
    [
        sg.Text("Breakfast", size = (38, 1), justification = 'center')
    ],
    [
        sg.Text("-", size = (38, 1), key = "bFood")
    ],
    [
        sg.Text("-", size = (38, 1), key = "bServing")
    ],
    [
        sg.Text("-", size = (38, 1), key = "bCalories")
    ],
    [
        sg.Text()
    ],
    [
        sg.Text("Lunch", size = (38, 1), justification = 'center')
    ],
    [
        sg.Text("-", size = (38, 1), key = "lFood")
    ],
    [
        sg.Text("-", size = (38, 1), key = "lServing")
    ],
    [
        sg.Text("-", size = (38, 1), key = "lCalories")
    ],
    [
        sg.Text()
    ],
    [
        sg.Text("Dinner", size = (38, 1), justification = 'center')
    ],
    [
        sg.Text("-", size = (38, 1), key = "dFood")
    ],
    [
        sg.Text("-", size = (38, 1), key = "dServing")
    ],
    [
        sg.Text("-", size = (38, 1), key = "dCalories")
    ],
    [
        sg.Text()
    ],
    [
        sg.Button('Update Menu', size = (38, 1), button_color = ('black', '#F6D180'))
    ]
]

layout = [
    [
        sg.Column(update_biodata, key = 'layout1', visible = False),
        sg.Column(todays_menu, key = 'layout2', visible = True)
    ]
]

window = sg.Window("H(EA)L(T)HY", layout, size = (350, 600), finalize = True)

healthy_icon = Image.open('healthy_icon.png').resize((50, 50), resample = Image.BICUBIC)
window['healthyIcon1'].update(data = ImageTk.PhotoImage(image = healthy_icon))
window['healthyIcon2'].update(data = ImageTk.PhotoImage(image = healthy_icon))

healthy = Image.open('healthy.png').resize((89, 50), resample = Image.BICUBIC)
window['healthy1'].update(data = ImageTk.PhotoImage(image = healthy))
window['healthy2'].update(data = ImageTk.PhotoImage(image = healthy))

while True:
    event, values = window.read()
    
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
        
    if event == "Generate Menu":
        uAge = float(values["uAge"])
        uWeight = float(values["uWeight"])
        uHeight = float(values["uHeight"])
        uSex = values["uSex"]
        uActivity = values["uActivity"]
        
        caloriesOutput = findCaloriesNeed(uAge, uWeight, uHeight, uSex, uActivity)
        breakfast = food.iloc[(food['Calories'].astype(int) - (caloriesOutput * 0.3)).abs().argsort()[:1]]
        lunch = food.iloc[(food['Calories'].astype(int) - (caloriesOutput * 0.5)).abs().argsort()[:1]]
        dinner = food.iloc[(food['Calories'].astype(int) - (caloriesOutput * 0.2)).abs().argsort()[:1]]
    
        window["dailyCalories"].update("Total Calories Need:" + str(caloriesOutput) + " cal")
        
        window["bFood"].update(breakfast['Food'].values[0])
        window["bServing"].update(breakfast['Serving'].values[0])
        window["bCalories"].update(breakfast['Calories'].values[0] + " cal")
        
        window["lFood"].update(lunch['Food'].values[0])
        window["lServing"].update(lunch['Serving'].values[0])
        window["lCalories"].update(lunch['Calories'].values[0] + " cal")
        
        window["dFood"].update(dinner['Food'].values[0])
        window["dServing"].update(dinner['Serving'].values[0])
        window["dCalories"].update(dinner['Calories'].values[0] + " cal")
        
        window[f'layout1'].update(visible = False)
        window[f'layout2'].update(visible = True)
    
    if event == "Update Menu":
        window[f'layout1'].update(visible = True)
        window[f'layout2'].update(visible = False)
        
    if event == "Cancel":
        window[f'layout1'].update(visible = False)
        window[f'layout2'].update(visible = True)
        
window.close()